In [3]:
%%writefile MakeCube.py

import sys, os, math, gzip, time
from panda3d.core import *

def vector_copy(v):
    """Utility function to coerce a list or tuple into Vec3 format"""
    return Vec3(v[0], v[1], v[2])

def point_copy(p):
    """Utility function to coerce a list or tuple into Point3 format"""    
    return Point3(p[0], p[1], p[2])
        
class myTextureStage(object):
    """Custom class to allow access to texturestage and texture data"""
    def __init__(self, name, ts, tex, mode, matname=""):
        self.ts = ts
        self.tex = tex
        self.name = name
        self.mode = mode
        self.matname = matname

    def getMode(self):
        return self.mode

    def getName(self):
        return self.name

    def getTextureStage(self):
        return self.ts

    def getTexture(self):
        return self.tex

    def getMatname(self):
        return self.matname

    def setTexture(self, texture):
        self.tex.load(texture)

class myGeom(object):
    def __init__(self, center=[0.0,0.0,0.0], scale=1.0, smooth=True, tangents=True, verts=None, polys=None, mats=None, norms=None, uvs=None, mapping="box", fit_uvs=False):
        """
        Variable mapping should be a string defining the UV mapping type to use.
        Options are:
        'planar' = Planar mapping along forward axis
        'box' = Box mapping
        'sphere' = Sphere mapping along vertical axis, with split poles
        'cylinder' = Cylinder mapping, no caps
        'cylinder_cap' or 'cylinder_caps' = Cylinder mapping with caps
        'global' = Global mapping, in which all objects mapped with this type will share the same UV space
        'tile' or 'poly' = Per-poly, tiled mapping, with each poly maximized to fill the entire 0-1 UV range 
        The object will not be mapped if texcoords are provided in the uvs list.
        
        Instantiate the class providing at least verts and polys, in the format shown below.
        Without both verts and polys submitted, class will default to creating an example box geometry.
        This class was intended to be compatible with e.g. a simple obj file reader, which would pass
        the vertices, polygons, normals, and materials from the obj.
        
        If smooth is True and norms is None, vertex normals will be smoothed across polygons.
        If tangents is True, tangents and bitangents will be calculated.
        """
        self.example = False
        self.smooth = smooth
        self.use_tangents = tangents
        self.mapping = mapping
        self.fit_uvs = fit_uvs
        if len(scale) == 1:
            scale = [scale, scale, scale]
        if type(scale) is not list:
            scale = list(scale)
            
        if verts:
            #self.verts = verts
            self.verts = [[0.0,0.0,0.0] for i in verts]
            for vi in range(len(verts)):
                for i in range(3):
                    self.verts[vi][i] = (verts[vi][i] + center[i]) * scale[i]
            if polys:
                self.polys = polys
            else:
                self.example = True
        else:
            self.example = True
            
        if self.example:    
            self.box = [center[0]-scale[0], center[0]+scale[0], 
                        center[1]-scale[1], center[1]+scale[1], 
                        center[2]-scale[2], center[2]+scale[2]]
            # box format is [minX,maxX,minY,maxY,minZ,maxZ]            
            self.verts = [ [self.box[0],self.box[2],self.box[4]], [self.box[0],self.box[3],self.box[4]],
                           [self.box[0],self.box[3],self.box[5]], [self.box[0],self.box[2],self.box[5]],
                           [self.box[1],self.box[2],self.box[4]],[self.box[1],self.box[3],self.box[4]],
                           [self.box[1],self.box[3],self.box[5]],[self.box[1],self.box[2],self.box[5]] ]
            self.polys = [[3,2,1,0],[4,5,6,7],[5,4,0,1],[6,5,1,2],[7,6,2,3],[4,7,3,0]]
            self.mats = ["right", "left", "bottom", "back", "top", "front"]
            
        if norms:
            self.vnorms = norms
        else:
            self.vnorms = None

        if uvs:
            self.uvs = uvs
            self.uvMapper = None
        else:
            self.uvs = None
            self.uvMapper = myUVMapper()

        if mats:
            self.mats = mats
        else:
            if not self.example:
                self.mats = ["Preview" for i in self.polys]

        if not self.smooth:
            self.split_verts()
            
        self.colors = []
        for name in self.mats:
            if not name in self.colors:
                self.colors.append(name)

        self.textureStages = None

    def split_verts(self):
        """Split all the polygons, for hard edges."""
        verts = [[j for j in i] for i in self.verts]
        self.verts = []
        polys = [[j for j in i] for i in self.polys]
        self.polys = [[] for i in polys]
        for poly_index in range(len(polys)):
            poly = polys[poly_index]
            for vi in poly:
                self.verts.append([i for i in verts[vi]])
                self.polys[poly_index].append(len(self.verts)-1)
        if (self.vnorms != None) and (len(self.vnorms) != len(self.verts)):
            self.vnorms = None # Clear the normals list if lengths no longer match

    def build(self):
        # http://www.panda3d.org/forums/viewtopic.php?t=11528
        """Create the geometry from the submitted arrays"""
        verts = self.verts
        polys = self.polys
        self.geomnode = GeomNode('geometry')
        self.color_lookup = []
        if not self.vnorms:            
            self.getNormals()
        if not self.uvs:            
            self.getUVMapping()
        if self.use_tangents:
            self.getTangents()

        # Build array for new format.
        array = GeomVertexArrayFormat()
        array.addColumn(InternalName.make('vertex'), 3, Geom.NTFloat32, Geom.CPoint)    
        array.addColumn(InternalName.make('texcoord'), 2, Geom.NTFloat32, Geom.CTexcoord)
        array.addColumn(InternalName.make('normal'), 3, Geom.NTFloat32, Geom.CVector)
        if self.use_tangents:
            array.addColumn(InternalName.make('binormal'), 3, Geom.NTFloat32, Geom.CVector)
            array.addColumn(InternalName.make('tangent'), 3, Geom.NTFloat32, Geom.CVector)

        # Create and register format.
        format = GeomVertexFormat()
        format.addArray(array)
        format = GeomVertexFormat.registerFormat(format)

        geoms = []
        for i in range(len(self.colors)):
            vdata = GeomVertexData('ngon', format, Geom.UHStatic)
            geom = Geom(vdata)
            tri = GeomTriangles(Geom.UHStatic)
            vertex = GeomVertexWriter(vdata, 'vertex')
            normal = GeomVertexWriter(vdata, 'normal')
            texcoord = GeomVertexWriter(vdata, 'texcoord')            
            geoms.append({'geom':geom,
                          'tri':tri,
                          'vertex':vertex,
                          'texcoord':texcoord,
                          'normal':normal,
                          'index':0,
                          'vdata':vdata,
                          'color':i})
            if self.use_tangents:
                tangent = GeomVertexWriter(vdata, 'tangent')
                binormal = GeomVertexWriter(vdata, 'binormal')
                geoms[i]['tangent'] = tangent
                geoms[i]['binormal'] = binormal 
        
        for poly_index in range(len(polys)):
            color_index = self.colors.index(self.mats[poly_index])  
            vertcount = geoms[color_index]['index']            
            
            p = polys[poly_index]
            poly = [verts[i] for i in p]
            uvs = self.uvs[poly_index]       
            norm = [self.vnorms[i] for i in p]
            if self.use_tangents:
                binorm = [self.binorms[i] for i in p]
                tan = [self.tans[i] for i in p]
            reindexed = [] # New vertex indices per-poly
        
            for v in poly:
                geoms[color_index]['vertex'].addData3f(v[0], v[1], v[2])
                reindexed.append(vertcount)
                vertcount += 1
            geoms[color_index]['index'] = vertcount             
            for i in range(len(poly)):
                geoms[color_index]['normal'].addData3f(Vec3(norm[i][0], norm[i][1], norm[i][2]))
                if self.use_tangents:
                    geoms[color_index]['binormal'].addData3f(Vec3(binorm[i][0], binorm[i][1], binorm[i][2]))
                    geoms[color_index]['tangent'].addData3f(Vec3(tan[i][0], tan[i][1], tan[i][2]))
            for tvert in uvs:
                geoms[color_index]['texcoord'].addData2f(tvert[0], tvert[1])

            triangulated = self.getFanning(reindexed) # Use new vertex indices
            for tri_index in range(len(triangulated)):
                t = triangulated[tri_index]
                tri = geoms[color_index]['tri']
                tri.addVertices(t[0], t[1], t[2])
                
        for color_index in range(len(self.colors)):
            geom = geoms[color_index]['geom']
            tri = geoms[color_index]['tri']
            tri.closePrimitive()            
            geom.addPrimitive(tri)        
            self.geomnode.addGeom(geom)
            self.color_lookup.append(color_index)
        self.setMats()    

    def setMats(self): # This shows how to set up multi-texturing on a procedural geometry.
        """Set the texture and normal maps"""
        color_attribs = []
        self.textureStages = []        
        for i in range(len(self.colors)):            
            name = self.colors[i]            
            tex = Texture("%s_tex" %(name))              
            texAttrib = TextureAttrib.make()            
            ts_tex = TextureStage("%s_tex"%(name))
            ts_tex.setMode(TextureStage.MModulate)                
            texAttrib = texAttrib.addOnStage(ts_tex, tex)
            bump = Texture("%s_bump" %(name))
            ts_bump = TextureStage("%s_bump"%(name))
            ts_bump.setMode(TextureStage.MNormal)
            texAttrib = texAttrib.addOnStage(ts_bump, bump)
            color_attribs.append(texAttrib)
            self.textureStages.append(myTextureStage("%s_tex"%(name), ts_tex, tex, TextureStage.MModulate, matname=name))
            self.textureStages.append(myTextureStage("%s_bump"%(name),ts_bump, bump, TextureStage.MNormal, matname=name))        
        for index in range(self.geomnode.getNumGeoms()):
            color = self.color_lookup[index]
            texAttrib = color_attribs[color]            
            newRenderState = self.geomnode.getGeomState(index).addAttrib(texAttrib,1)
            self.geomnode.setGeomState(index, newRenderState)

    def getUVMapping(self):
        """Get the UV mapping from the myUVMapper class instance"""        
        if self.mapping == "box":
            self.uvMapper.box_map(self.verts, self.polys, inbounds=0)
        elif self.mapping == "planar":
            self.uvMapper.planar_map(self.verts, self.polys, axis=1)
        elif self.mapping == "sphere":
            self.uvMapper.sphere_map(self.verts, self.polys, axis=2, split_poles=1, fix_overlap=1)
        elif self.mapping == "cylinder":
            self.uvMapper.cylinder_map(self.verts, self.polys, axis=2, fix_overlap=1)
        elif self.mapping == "cylinder_caps" or self.mapping == "cylinder_cap":            
            self.uvMapper.cylinder_map_caps(self.verts, self.polys, axis=1, fix_overlap=1)
        elif self.mapping == "global":
            self.uvMapper.global_map(self.verts, self.polys)
        elif self.mapping == "tile" or self.mapping== "poly":
            self.uvMapper.tile_map(self.verts, self.polys)
        if self.fit_uvs:
            self.uvMapper.fitUVs()
        self.uvs = self.uvMapper.uvs

    def normalize(self, myVec):
        """Convenience function, from Panda Procedural Cube example."""
        myVec.normalize()
        return myVec

    def getNorms(self, v1, v2, v3):
        """Get the poly normal for a tri/poly/Ngon"""
        e1 = self.normalize(v1 - v2)
        e2 = self.normalize(v3 - v2)
        norm = self.normalize(e2.cross(e1))
        return norm

    def getFanning(self, poly):
        """Split Ngon into tris, using simple fanning method"""
        triangulated = []
        for index in range(1,len(poly)-1): # Number of tris per poly will be number of verts in poly, minus 2            
            triangulated.append([poly[0], poly[index], poly[index+1]]) 
        return triangulated
        
    def getTangents(self):
        """
        Calculate tangents and bitangents (binormals).
        Requires both normals and texcoords up front.
        We need a set of orthonormal vectors for the normal, but
        these need to be properly aligned with the uvs for each poly.
        """
        # http://gamedev.stackexchange.com/questions/68612/how-to-compute-tangent-and-bitangent-vectors
        verts = self.verts
        polys = self.polys
        uvs = self.uvs
        norms = self.vnorms
        tan1 = [Vec3(0,0,0) for i in verts]
        tan2 = [Vec3(0,0,0) for i in verts]
        self.tans = [None for i in verts]
        self.binorms = [None for i in verts]
        epsilon = 1.19209290e-010
        for poly_index in range(len(polys)):
            p = polys[poly_index]
            i1 = p[0]
            i2 = p[1]
            i3 = p[2]
            
            v1 = verts[i1]
            v2 = verts[i2]
            v3 = verts[i3]

            #print uvs[poly_index]
            w1 = uvs[poly_index][0]
            w2 = uvs[poly_index][1]
            w3 = uvs[poly_index][2]

            x1 = v2[0] - v1[0]
            x2 = v3[0] - v1[0]
            y1 = v2[1] - v1[1]
            y2 = v3[1] - v1[1]
            z1 = v2[2] - v1[2]
            z2 = v3[2] - v1[2]

            s1 = w2[0] - w1[0]
            s2 = w3[0] - w1[0]
            t1 = w2[1] - w1[1]
            t2 = w3[1] - w1[1]

            d = (s1 * t2 - s2 * t1)
            if abs(d) < epsilon: # Float division error protection.
                d = epsilon                
            r = 1.0 / d
            sdir = Vec3((t2 * x1 - t1 * x2) * r, (t2 * y1 - t1 * y2) * r, (t2 * z1 - t1 * z2) * r)
            tdir = Vec3((s1 * x2 - s2 * x1) * r, (s1 * y2 - s2 * y1) * r, (s1 * z2 - s2 * z1) * r)
            
            tan1[i1] += sdir
            tan1[i2] += sdir
            tan1[i3] += sdir

            tan2[i1] += tdir
            tan2[i2] += tdir
            tan2[i3] += tdir            

        for a in range(len(verts)):            
            n = norms[a]
            t = tan1[a]            
            # Gram-Schmidt orthogonalize
            self.tans[a] = self.normalize((t - n * n.dot(t)))
            # Calculate handedness
            ncross = n.cross(t)
            t2 = Vec3(tan2[a][0], tan2[a][1], tan2[a][2])
            if ncross.dot(t2) < 0.0: # If pointing in opposite directions
                self.tans[a] *= -1.0 # Flip it            
            self.binorms[a] = self.normalize(self.tans[a].cross(n))            
    
    def getNormals(self):
        """
        Gathers all the face normals, then the vertex normals from those.
        """
        self.pnorms = [None for i in range(len(self.polys))]
        polys = self.polys
        for poly_index in range(len(polys)):
            p = polys[poly_index]
            face = [self.verts[i] for i in p]
            vecs = []
            for v in face:               
                vecs.append(Vec3(v[0], v[1], v[2]))
            self.pnorms[poly_index] = self.getNorms(vecs[0], vecs[1], vecs[2])
        self.getVertpolys()        
        self.vnorms = self.getVertexNormals(self.pnorms)

    def getVertpolys(self):
        """
        self.vpolys is a convenience array to assist with sorting.
        vpolys = polygons of vertices
        Whereas self.polys lists verts of polys, this lists polys of verts.
        Used here to sort polygon normals into vertex normals.
        """
        polys = self.polys
        self.vpolys = [[] for i in range(len(self.verts))]
        for pvi in range(len(polys)):
            for v in polys[pvi]:
                if not pvi in self.vpolys[v]:
                    self.vpolys[v].append(pvi)

    def getVertexNormals(self, norms):
        """
        norms is poly normals, binormals, or tangents
        vpolys is polygons of vertices, from myMesh instance 
        Get the vertex normals from poly normals
        """
        vpolys = self.vpolys
        vnorms = [[0.0,0.0,0.0] for i in vpolys]
        for vi in range(len(vpolys)):
            polys = vpolys[vi]
            nx = 0; ny = 0; nz = 0
            l = len(polys)            
            if l == 0: # In some unusual geometries, there may be vertices with no polygon associations - we try to skip these.  1/26/08                
                #print "Stray vertex: %s, %s" %(vi,l)
                continue # If we don't skip these, we get a divide by zero crash error due to a bad mesh.
            for p in polys:
                nx += norms[p][0]
                ny += norms[p][1]
                nz += norms[p][2]
            n = Vec3(nx/l,ny/l,nz/l)
            n = self.normalize(n)    # Shouldn't need to do this, but it can't hurt        
            vnorms[vi] = n
        return vnorms

class myUVMapper(object):
    def __init__(self):        
        self.uvs = []
        self.epsilon = 1.19209290e-07

    def getAxis(self, xdir):
        """Get the major axis along which a vector points"""
        absz = [abs(i) for i in xdir]
        index = absz.index(max(absz))
        sign = self.getSign(xdir[index])
        axis = [0,0,0]
        axis[index] = 1 * sign    
        return axis

    def getSign(self, num):
        if abs(num) == num:
            return 1
        return -1

    def getBounds(self, verts):
        """
        create bounding boxes
        """
        # box format is [minX,maxX,minY,maxY,minZ,maxZ]
        minX = 1000000.0; maxX = -1000000.0
        minY = 1000000.0; maxY = -1000000.0
        minZ = 1000000.0; maxZ = -1000000.0
        for i in range(len(verts)):
            tx = verts[i][0]
            ty = verts[i][1]
            tz = verts[i][2]
            if tx < minX: minX = tx
            if tx > maxX: maxX = tx
            if ty < minY: minY = ty
            if ty > maxY: maxY = ty
            if tz < minZ: minZ = tz
            if tz > maxZ: maxZ = tz
        return [minX,maxX,minY,maxY,minZ,maxZ]

    def box_center(self, box):
        """
        find the center of the box
        To use results as a cylinder: use box_center x,z with vertex y
        Game gems: can also do (min + max)/2 for each axis.
        """
        minX,maxX,minY,maxY,minZ,maxZ = box
        cx = (abs(minX-maxX)/2)+minX
        cy = (abs(minY-maxY)/2)+minY
        cz = (abs(minZ-maxZ)/2)+minZ
        return [cx,cy,cz]

    def normalize(self, myVec):
        myVec.normalize()
        return myVec

    def getUVBounds(self, uvs, single=False):
        """
        create bounding boxes
        """
        # box format is [minX,maxX,minY,maxY]
        minX = 1000000.0; maxX = -1000000.0
        minY = 1000000.0; maxY = -1000000.0
        if single:
            uvs = [uvs]
        for uvset in uvs:
            for i in range(len(uvset)):
                tx = uvset[i][0]
                ty = uvset[i][1]            
                if tx < minX: minX = tx
                if tx > maxX: maxX = tx
                if ty < minY: minY = ty
                if ty > maxY: maxY = ty            
        return [minX,maxX,minY,maxY]

    def fitUVs(self):
        bounds = self.getUVBounds(self.uvs)        
        udist = bounds[0]
        vdist = bounds[2]
        for poly in self.uvs:
            for uv in poly:                
                uv[0] -= udist
                uv[1] -= vdist 
        bounds = self.getUVBounds(self.uvs)        
        for poly in self.uvs:
            for uv in poly:
                uv[0] = uv[0]/bounds[1]
                uv[1] = uv[1]/bounds[3]

    def fitUVSingle(self,poly):
        bounds = self.getUVBounds(poly,single=1)
        udist = bounds[0]
        vdist = bounds[2]
        for uv in poly:                
            uv[0] -= udist
            uv[1] -= vdist
        bounds = self.getUVBounds(poly,single=1)
        for uv in poly:
            uv[0] = uv[0]/bounds[1]
            uv[1] = uv[1]/bounds[3]

    def tile_map(self,verts,polys):
        self.box_map(verts,polys)
        for poly in self.uvs:
            self.fitUVSingle(poly)
        

    def planar_map(self, verts, polys, axis=1):
        """Create a planar map if there are no UVs.  Uses Panda Y (forward) axis by default."""
        bbox = self.getBounds(verts)
        if axis == 0:
            minx = bbox[2]
            maxx = bbox[3]
            miny = bbox[4]
            maxy = bbox[5]
            index1 = 1
            index2 = 2
        elif axis == 1:
            minx = bbox[0]
            maxx = bbox[1]
            miny = bbox[4]
            maxy = bbox[5]            
            index1 = 0
            index2 = 2
        elif axis == 2:
            minx = bbox[0]
            maxx = bbox[1]
            miny = bbox[2]
            maxy = bbox[3]            
            index1 = 0
            index2 = 1
        sizex = max(maxx - minx, 1)        
        sizey = max(maxy - miny, 1)
        self.uvs = [[] for i in range(len(polys))]
        for poly_index in range(len(polys)):
            poly = polys[poly_index]
            for vi in poly:
                vert = verts[vi]
                u = (vert[index1] - minx)/sizex
                v = (vert[index2] - miny)/sizey
                self.uvs[poly_index].append([u,v])        

    def box_map(self,verts,polys,inbounds=0):
        """
        Apply box-mapped UVs.
        Adapted from UV-mapping plugin for AC3D.
        http://www.inivis.com/supercoldmilk/ac3dplug/uvmap.html
        """
        bounds = self.getBounds(verts)
        nx,mx,ny,my,nz,mz = bounds
        szx = max(mx - nx,1)
        szy = max(my - ny,1)
        szz = max(mz - nz,1)
        self.uvs = []        
        for pi in range(len(polys)):
            psets = polys[pi]            
            edge1 = point_copy(verts[psets[1]]) - point_copy(verts[psets[0]])            
            edge2 = point_copy(verts[psets[0]]) - point_copy(verts[psets[len(psets)-1]])
            norm = edge1.cross(edge2)
            norm = self.normalize(norm)
            axis = self.getAxis(norm)
            uvs = []
            for vi in psets:            
                vert = verts[vi]                
                if axis[0]:
                    u = (vert[2] - nz)/szz
                    v = (vert[1] - ny)/szy
                    u = -u
                    if norm[0] > 0.0:
                        u = -u
                elif axis[1]:
                    u = (vert[0] - nx)/szx
                    v = (vert[2] - nz)/szz
                    u = -u
                    v = -v
                    if norm[1] < 0.0:
                        u = -u
                else:
                    u = (vert[0] - nx)/szx
                    v = (vert[1] - ny)/szy
                    u = -u                
                    if norm[2] < 0.0:
                        u = -u
                if inbounds:
                    u *= 0.5
                    v *= 0.5
                    u += 0.5
                    v += 0.5
                uvs.append([u,v])
            self.uvs.append(uvs)        

    def cylinder_map(self,verts,polys,axis=2,fix_overlap=0):
        """
        Adapted from UV-mapping plugin for AC3D.
        http://www.inivis.com/supercoldmilk/ac3dplug/uvmap.html
        Create a cylindrical map if there are no UVs.
        Does not split poles/handle caps/split seams.
        """
        bounds = self.getBounds(verts)
        nx,mx,ny,my,nz,mz = bounds
        center = self.box_center(bounds)
        if axis == 0:
            north = [1.0,0.0,0.0]
            equator = [0.0,0.0,1.0]
            selheight = mx - nx
        elif axis == 2:
            north = [0.0,0.0,1.0]
            equator = [0.0,1.0,0.0]
            selheight = mz - nz
        else:
            north = [0.0,1.0,0.0]
            equator = [0.0,0.0,1.0]
            selheight = my - ny
        north = point_copy(north)
        equator = point_copy(equator)
        northequatorx = north.cross(equator)
        self.uvs = []
        
        for pi in range(len(polys)):
            psets = polys[pi]
            uvs = []
            for vi in psets:            
                vert = point_copy(verts[vi])
                vcenter = point_copy(center)
                vcenter[axis] = vert[axis]
                vray = vert - vcenter
                vray = self.normalize(vray)
                phi = math.acos(north.dot(vray))
                if ((vray[axis] == 1.0) or (vray[axis] == -1.0)):            
                    u = 0.5
                else:
                    u = math.acos(max(min(equator.dot(vray)/max(math.sin(phi),self.epsilon),1.0),-1.0)) / (2.0*math.pi + 0.5)
                    #if northequatorx.dot(vray) < 0.0: u = -u
                if axis == 0:
                    v = (vert[0] - nx)/selheight
                elif axis == 2:
                    v = (vert[2] - nz)/selheight
                else:
                    v = (vert[1] - ny)/selheight
                uvs.append([(u+0.5),v]) #uvs.append([(u+0.5),v])
            self.uvs.append(uvs)
            if fix_overlap:
                self.FixUVWrap(pi)

    def cylinder_map_caps(self,verts,polys,axis=2,fix_overlap=0):
        """
        Adapted from UV-mapping plugin for AC3D.
        http://www.inivis.com/supercoldmilk/ac3dplug/uvmap.html
        Create a cylindrical map if there are no UVs.        
        """
        bounds = self.getBounds(verts)
        nx,mx,ny,my,nz,mz = bounds
        szx = max(mx - nx,1)
        szy = max(my - ny,1)
        szz = max(mz - nz,1)
        center = self.box_center(bounds)
        if axis == 0:
            north = [1.0,0.0,0.0]
            equator = [0.0,0.0,1.0]
            selheight = mx - nx
        elif axis == 2:
            north = [0.0,0.0,1.0]
            equator = [0.0,1.0,0.0]
            selheight = mz - nz
        else:
            north = [0.0,1.0,0.0]
            equator = [0.0,0.0,1.0]
            selheight = my - ny
        north = point_copy(north)
        equator = point_copy(equator)
        northequatorx = north.cross(equator)
        self.uvs = []
        
        for pi in range(len(polys)):            
            psets = polys[pi]
            uvs = []
            fixuvs = False
            edge1 = point_copy(verts[psets[1]]) - point_copy(verts[psets[0]])            
            edge2 = point_copy(verts[psets[0]]) - point_copy(verts[psets[len(psets)-1]])
            norm = edge1.cross(edge2)
            norm = self.normalize(norm)
            plane = self.getAxis(norm)
            for vi in psets:            
                vert = point_copy(verts[vi])
                if plane[0] and axis == 0:
                    u = (vert[2] - nz)/szz
                    v = (vert[1] - ny)/szy  
                elif plane[1] and axis == 1:
                    u = (vert[0] - nx)/szx
                    v = (vert[2] - nz)/szz   
                elif plane[2] and axis == 2:
                    u = (vert[0] - nx)/szx
                    v = (vert[1] - ny)/szy
                else:
                    fixuvs = True                
                    vcenter = point_copy(center)
                    vcenter[axis] = vert[axis]
                    vray = vert - vcenter
                    vray = self.normalize(vray)
                    phi = math.acos(north.dot(vray))
                    if ((vray[axis] == 1.0) or (vray[axis] == -1.0)):            
                        u = 0.5
                    else:
                        u = math.acos(max(min(equator.dot(vray)/max(math.sin(phi),self.epsilon),1.0),-1.0)) / (2.0*math.pi) + 0.5
                        #if northequatorx.dot(vray) > 0.0: u = 1.0 - u
                    if axis == 0:
                        v = (vert[0] - nx)/selheight
                    elif axis == 2:
                        u = 1.0 - u
                        v = (vert[2] - nz)/selheight
                    else:
                        v = (vert[1] - ny)/selheight                
                uvs.append([u,v])           
            self.uvs.append(uvs)
            if fix_overlap and fixuvs:
                self.FixUVWrap(pi)                

    def sphere_map(self,verts,polys,axis=2,fix_overlap=0,split_poles=0):
        """
        Create a spherical map if there are no UVs.
        http://www.inivis.com/supercoldmilk/ac3dplug/uvmap.html
        Doesn't split texverts at edges.
        """
        if axis == 0:
            north = [1,0,0]
            equator = [0,0,1]
        if axis == 1:
            north = [0,0,1]
            equator = [0,1,0]
        if axis == 2:
            north = [0,1,0]
            equator = [0,0,1]
        north = vector_copy(north)
        equator = vector_copy(equator)
        northequatorx = north.cross(equator)    
        bbox = self.getBounds(verts)
        center = point_copy(self.box_center(bbox))
        self.uvs = [[] for i in range(len(polys))]
        for poly_index in range(len(polys)):
            poly = polys[poly_index]
            for vi in poly:
                vert = point_copy(verts[vi])
                vray = vert - center
                vray = self.normalize(vray)
                phi = math.acos(north.dot(vray))
                v = 1.0 - (phi/math.pi)                
                if axis == 0 and (vray[0] == 1.0 or vray[0] == -1.0):
                    u = 0.5
                elif axis == 2 and (vray[1] == 1.0 or vray[1] == -1.0):
                    u = 0.5
                elif axis == 1 and (vray[2] == 1.0 or vray[2] == -1.0):
                    u = 0.5
                else:                    
                    u = math.acos(max(min(equator.dot(vray)/max(math.sin(phi),self.epsilon),1.0),-1.0)) / (2.0*math.pi) + 0.5                    
                    #if northequatorx.dot(vray) > 0.0: u = 1.0 - u
     
                self.uvs[poly_index].append([u,v])
            if fix_overlap:
                self.FixUVWrap(poly_index)
        if split_poles:            
            self.SpacePolarCoordinates(polys, verts, axis, self.box_center(bbox))

    def FixUVWrap(self, poly_index):
        #===============================================================
        # Courtesy Andy Colebourne, with edits to get min/max tu:
        # check for overlaps in spherical mapping
        flag = 0
        uvs = self.uvs
        poly = self.uvs[poly_index]           
        for uv1 in poly:
            for uv2 in poly:
                if 0.5 <= math.fabs(uv1[0] - uv2[0]):
                    flag |= 1

        minu = 100.0; maxu = -100.0
        if flag == 1:       

            for uvi in range(len(poly)):
                uv = poly[uvi]
                if uv[0] < 0.5:
                    self.uvs[poly_index][uvi][0] += 1.0
                    uv[0] += 1.0
                    #print poly_index, uvi
                minu = min(uv[0], minu)
                maxu = max(uv[0], maxu)
        #===============================================================
        # Pull texture coordinates back if over 1/2 of the horizontal bounds is past 1.0
        if 1.0 - minu < maxu - 1.0:
            for uvi in range(len(poly)):
                uv = poly[uvi]
                self.uvs[poly_index][uvi][0] -= 1.0

    def SpacePolarCoordinates(self, polys, verts, axis, vcenter):        
        for poly_index in range(len(polys)):
            poly = polys[poly_index]
            uvs = self.uvs[poly_index]
            for uvi in range(len(poly)):
                v = verts[poly[uvi]]
                polar = False                
                if axis == 0:
                    # X axis
                    if math.fabs(vcenter[1] - v[1]) < self.epsilon and math.fabs(vcenter[2] - v[2]) < self.epsilon:
                        polar = True
                elif axis == 1: # Y and Z need to be flipped for Panda compatability -- here, but not in sphere_map.
                    # Z axis
                    if math.fabs(vcenter[1] - v[1]) < self.epsilon and math.fabs(vcenter[0] - v[0]) < self.epsilon:
                        polar = True
                else:
                    # Y axis
                    if math.fabs(vcenter[0] - v[0]) < self.epsilon and math.fabs(vcenter[2] - v[2]) < self.epsilon:
                        polar = True
                if polar:
                    # Move the horizontal UV to the unweighted center            
                    first = True
                    for uvi2 in range(len(poly)):
                        if uvi2 == uvi:
                            continue
                        uv = uvs[uvi2]
                        if first:
                            tumin = uv[0]
                            tumax = uv[0]
                            first = False
                        else:
                            tumin = min(tumin,uv[0])
                            tumax = max(tumax,uv[0])
                        # Ensure we read at least one entry (!first)
                        if not first:
                            self.uvs[poly_index][uvi][0] = tumin + ((tumax - tumin) * 0.5)


    def global_map(self,verts,polys,scale=1.0,offset=0.0,rotation=0.0):
        # Front UVs are flipped?
        """
        Adapted from UV-mapping plugin for AC3D.
        http://www.inivis.com/supercoldmilk/ac3dplug/uvmap.html
        """
        self.uvs = []
        scale = 1.0/scale
        rotating = math.fabs(rotation) > 0.00001
        if rotating:
            rotcos = math.cos( math.radians(rotation))
            rotsin = math.sin( math.radians(rotation))
        for pi in range(len(polys)):
            psets = polys[pi]            
            edge1 = point_copy(verts[psets[1]]) - point_copy(verts[psets[0]])
            edge2 = point_copy(verts[psets[0]]) - point_copy(verts[psets[len(psets)-1]])
            norm = edge1.cross(edge2)
            norm = self.normalize(norm)        
            normx = 1
            normy = 1
            axis = self.getAxis(norm)
            ptv = []
            uvs = []            
            for vi in psets:            
                vert = verts[vi]
                if axis[0]:
                    u = vert[2]
                    v = vert[1]
                    if norm[0] > 0.0:
                        u = -u
                elif axis[1]:
                    u = vert[0]
                    v = vert[2]
                    if norm[1] < 0.0:
                        v = -v                        
                else:
                    u = vert[0]
                    v = vert[1]
                    if norm[2] < 0.0:
                        u = -u                        
                if (u < 1.0) and (u > -1.0):
                    normx = 0
                if (v < 1.0) and (v > -1.0):
                    normy = 0
                if rotating:
                    tvx = u  # keep an uncorrupted copy
                    u = (u * rotcos) - (tverty * rotsin)
                    v = (tvx * rotsin) + (tverty * rotcos)
                u = (u * scale) + offset
                v = (v * scale) + offset
                ptv.append([u,v])            
            if normx or normy:                
                absx = 0.0
                u = ptv[0][0]
                if u > 1.0:
                    absx = math.floor(u)
                else:
                    absx = math.ceil(u)
                absy = 0.0
                v = ptv[0][1]
                if v > 1.0:
                    absy = math.floor(v)
                else:
                    absy = math.ceil(v)            
                for uvi in range(1,len(ptv)):
                    u,v = ptv[uvi]
                    if normx:
                        if math.fabs(u) < math.fabs(absx):
                            if u > 1.0:
                                absx = math.floor(u)
                            else:
                                absx = math.ceil(u)
                    if normy:
                        if math.fabs(v) < math.fabs(absy):
                            if v > 1.0:
                                absy = math.floor(v)
                            else:
                                absy = math.ceil(v)
                for uvi in range(len(ptv)):
                    ptv[uvi][0] -= absx
                    ptv[uvi][1] -= absy
            for u,v in ptv:
                #uvs.append([-(u+0.5)*0.5,v*0.5])
                #uvs.append([-(u+0.5),v])
                uvs.append([u,v])
            self.uvs.append(uvs)

class myTextures(object):
    def __init__(self, path="models/",colors=None, names=None, verbose=False):
        if colors:
            self.colors = colors
        else:
            self.colors = [(185, 0, 0),
                           (81, 190, 0),
                           (0, 99, 192),
                           (187, 181, 0),
                           (96, 0, 186),
                           (187, 97, 0),
                           (35, 35, 35),
                           (220, 220, 220)]
        if names:
            self.colornames = names
        else:
            self.colornames = ['red','green','blue','yellow','purple','orange','black','white']            
        self.images = {}
        if not os.path.exists(path):
            path = ""
        else:
            path = path.replace("/",os.sep)
        self.path = path
        self.xsize = 0
        self.ysize = 0        
        self.normal_map = None
        self.verbose = verbose
        
    def makeTextures(self, xsize=240, ysize=240, splits=8, write=True):
        """Generate some PNMimages to apply as textures"""
        self.xsize = xsize
        self.ysize = ysize
        xdiv = xsize/splits
        ydiv = ysize/splits        
        colors = self.colors
        path = self.path
        path.replace("/", os.sep)        
        for i in range(len(colors)):
            color = colors[i]
            name = self.colornames[i]
            texture_path = os.path.join(path, "%s_tile.png" %(name))
            if write and os.path.exists(texture_path):
                if self.verbose: print("Texture %s already exists." %(name))
                continue
            if self.verbose: print("Generating texture %s..." %(name))
            if name == "white":
                color2 = (0, 0, 0)
            else:
                color2 = (255, 255, 255)
            img = PNMImage(xsize, ysize)
            for w in range(xsize):
                for h in range(ysize):
                    if w % 30 == 0 or (w+1) % 30 == 0 or (w-1) % 30 == 0 or\
                       h % 30 == 0 or (h+1) %30 == 0 or (h-1) % 30 == 0:
                        img.setXelVal(w,h,color2)
                    else:
                        img.setXelVal(w,h,color)
            img.gaussianFilter()
            img.boxFilter()
            if write:
                img.write(texture_path)
            else:
                self.images[name] = img            

    def makeHeight(self, img, filter="Sobel", channels=False):
        """Create the normal map.  From height2bump.py, under open license"""
        # http://www.pythonstuff.org/glsl/media/height2bump.py
        # http://www.pythonstuff.org/glsl/normalmaps_from_heightmaps.html
        xsize = img.getXSize()
        ysize = img.getYSize()
        
        if filter == "Sobel":
            a1=1; a2=2; a3=0
            b1=1; b2=4; b3=6
        elif filter == "Scharr":           # 5x5 opt Scharr Filter from
            a1=21.38; a2=85.24; a3=0       # http://nbn-resolving.de/urn/resolver.pl?urn=urn:nbn:de:bsz:16-opus-9622
            b1= 5.96; b2=61.81; b3=120.46
        else:
            raise ValueError( "Unknown 'filter' argument '" + filter + "'" )
        a4=-a2; a5=-a1
        b4=b2;  b5=b1
        kernel = []
        kernel.append((a1*b1, a2*b1, a3*b1, a4*b1, a5*b1,
                       a1*b2, a2*b2, a3*b2, a4*b2, a5*b2,
                       a1*b3, a2*b3, a3*b3, a4*b3, a5*b3,
                       a1*b4, a2*b4, a3*b4, a4*b4, a5*b4,
                       a1*b5, a2*b5, a3*b5, a4*b5, a5*b5)) # x
        kernel.append((b1*a1, b2*a1, b3*a1, b4*a1, b5*a1,
                       b1*a2, b2*a2, b3*a2, b4*a2, b5*a2,
                       b1*a3, b2*a3, b3*a3, b4*a3, b5*a3,
                       b1*a4, b2*a4, b3*a4, b4*a4, b5*a4,
                       b1*a5, b2*a5, b3*a5, b4*a5, b5*a5)) # y
        # to get the scale factor, we look at the extreme case: vertical fall from 255 to 0:        
        scale = 0.0
        for i, val in enumerate( kernel[0] ):
            if i % 5 < 5//2:
                scale += 255 * val
        scale /= 128
        
        r ,g ,b = self.split_channels(img) # Split into channels.  Each channel contains only one value, on
                                           # the assumption that any pixel in a gray channel contains only
                                           # the same value three times. 
        
        rr = self.imgfilter(r, kernel[0], 5, 128, scale) # Red handles x
        gg = self.imgfilter(g, kernel[1], 5, 128, scale) # Green handles y
        bb = [[128 for h in range(ysize)] for w in range(xsize)] # Red+green+blue should equal one   
        for y in range(xsize): # Calculate blue as the remainder, once red and green are factored in
            for x in range(ysize):
                op = 1.0 - (rr[x][y]*2.0/255 - 1.0)**2 - (gg[x][y]*2.0/255 - 1.0)**2
                if op > 0.0:
                    bb[x][y] = 128 + 128 * math.sqrt(op)
                else:
                    bb[x][y] = 128
        if channels:
            self.make_PNMImage("bump_red_channel", r=rr, write=True) # Show what the various channels are doing.
            self.make_PNMImage("bump_green_channel", g=gg, write=True)
            self.make_PNMImage("bump_blue_channel", b=bb, write=True)     
        return (rr, gg, bb)		

    def imgfilter(self, img, kernel, ker_size, offset, scale):
        """Drive the filter function"""
        iscale = 1.0/scale # Do the division up front and multiply later, for some (potential) small speed gain.   
        xsize = self.xsize
        ysize = self.ysize   
        new = [[j for j in i] for i in img] # Create the array to be filled with new pixel values   
        new = self.pad_image(new, xsize, ysize) # Pad both images, to allow neighbor lookups beyond true x and y lengths
        img2 = self.pad_image(img, xsize, ysize)
        offsets = self.kernel_offset(size=int(ker_size)) # Generate kernel offsets for lookups
        ker_square = ker_size**2 # Calculate the range up front, for some (potential) small speed gain
        for x in range(xsize):
            for y in range(ysize):                
                acc = 0
                for i in range(ker_square): # Run the PIL process as a loop
                    ox, oy = offsets[i]                    
                    acc += int(img2[x+ox][y+oy]) * kernel[i]
                acc = acc * iscale + offset # Scale the values and provide an offset (offset=128 is used, here).
                if acc > 255: # Clamp the values (seems unnecessary, but PIL does this).
                    acc = 255
                elif acc < 0:
                    acc = 0
                new[x][y] = acc # Set the new pixel value
        new = self.crop_image(new, xsize, ysize) # Crop out the image padding, to restore original image dimensions
        return new

    def pad_image(self, img, xsize, ysize, add=2):
        """Pad out the image"""
        new = [[0 for h in range(ysize + add)] for w in range(xsize + add)]
        for w in range(xsize):
            for h in range(ysize):
                new[w][h] = img[w][h]
        for w in range(xsize, xsize + add):
            x = xsize-1
            for h in range(ysize):
                new[w][h] = img[x][h]
        for h in range(ysize, ysize + add):
            y = ysize-1
            for w in range(xsize):
                new[w][h] = img[w][y]
        return new

    def crop_image(self, img, xsize, ysize, sub=2):
        """Crop the padding from the image"""
        new = [[img[w][h] for h in range(ysize)] for w in range(xsize)]
        return new 

    def kernel_offset(self, size):
        """
        size = 5 gives:
        [[-2, 2], [-1, 2], [0, 2], [1, 2], [2, 2],
        [-2, 1], [-1, 1], [0, 1], [1, 1], [2, 1],
        [-2, 0], [-1, 0], [0, 0], [1, 0], [2, 0],
        [-2, -1], [-1, -1], [0, -1], [1, -1], [2, -1],
        [-2, -2], [-1, -2], [0, -2], [1, -2], [2, -2]] 
        """
        offset_range = range(-int(size/2),int(size/2)+1)
        offsets = []
        for h in offset_range:
            for w in offset_range:
                offsets.append([w,-h])
        return offsets            

    def split_channels(self, img):
        """Split image into RGB components"""
        w = img.getXSize()
        h = img.getYSize()
        r = [[0 for j in range(h)] for i in range(w)]
        g = [[0 for j in range(h)] for i in range(w)]
        b = [[0 for j in range(h)] for i in range(w)]
        for i in range(w):
            for j in range(h):
                r[i][j] = img.getRedVal(i,j)
                g[i][j] = img.getGreenVal(i,j)
                b[i][j] = img.getBlueVal(i,j)                
                #print i, j, img.getPixel(i, j)
        return r, g, b

    def make_PNMImage(self, name, r=[], g=[], b=[], write=True):
        """Combine RGB channels into a PNMimage"""
        img = PNMImage(self.xsize, self.ysize)
        if self.verbose: print("Generating image %s..." %(name))    
        for w in range(self.xsize):
            for h in range(self.ysize):
                if r:                    
                    img.setRedVal(w,h,int(r[w][h]))
                else:
                    img.setRedVal(w,h,0)
                if g:
                    img.setGreenVal(w,h,int(g[w][h]))
                else:
                    img.setGreenVal(w,h,0)
                if b:
                    img.setBlueVal(w,h,int(b[w][h]*2))
                else:
                    img.setBlueVal(w,h,0)
        if write:
            path = self.path
            path.replace("/", os.sep)
            texture_path = os.path.join(path, "%s.png" %(name))
            img.write(texture_path)
        return img

Writing MakeCube.py


In [3]:
"""
Simple procedural cube example

This script contains classes for geometry creation, UV mapping, automated texture and normal map generation, and .obj file import,
as well as a utility class to allow greater access to TextureStage data.
This script illustrates:
- How to generate tangents and binormals at runtime
- How to create a geometry using a more extensible approach than is seen in the standard Panda Procedural Cube example
- How to apply multiple textures to one geomNode, created procedurally
- How to generate a normal map at runtime

Code adapted from cdrei's.
this example is intended to be free for any use.
"""


from panda3d.core import *
from direct.showbase.ShowBase import ShowBase
from direct.task import Task

import sys, os, math, gzip, time
from MakeCube import *
import json
import io

from PIL import Image
from math import sin, cos, pi

loadPrcFileData("","sync-video #f")
class Application(ShowBase):
    def __init__(self):
        
        #Window size and position
        loadPrcFileData("", "win-origin {} {}".format(0,0))
        loadPrcFileData("", "win-size {} {}".format(1280,720))
        loadPrcFileData("", "undecorated 1")
        
        ShowBase.__init__(self)
        #self.accept("escape", sys.exit)
        render.setAntialias(AntialiasAttrib.MAuto)
        render.setShaderAuto()
        base.setFrameRateMeter(True)

#         self.cam.setY(-10)

        # Use myTextures class to generate some textures and a corresponding normal map.
        self.texts = myTextures(path="")
        self.texts.makeTextures(write=False)
        
        white = self.texts.images["white"]
        r,g,b = self.texts.makeHeight(white)
        self.texts.normal_map = self.texts.make_PNMImage("test_bump", r=r, g=g, b=b, write=False)       
        
        def quit():
            taskMgr.remove('moveCharacter')
            self.destroy()
            self.userExit()
            self.finalizeExit()
            
        self.accept('q', quit)
        
        self.setupFisheye()
        self.setupEnv()
        self.setupLight()
        
        self.keyMap = {"left":0, "right":0, "forward":0, "backward":0} 
        
        self.accept("arrow_up", self.setKey, ["forward",1])
        self.accept("arrow_down", self.setKey, ["backward",1])
        self.accept("arrow_up-up", self.setKey, ["forward",0])
        self.accept("arrow_down-up", self.setKey, ["backward",0])
        
        self.accept("arrow_left", self.setKey, ["left",1])
        self.accept("arrow_right", self.setKey, ["right",1])
        self.accept("arrow_left-up", self.setKey, ["left",0])
        self.accept("arrow_right-up", self.setKey, ["right",0])
        
        self.prevtime = 0
        
        self.taskMgr.add(self.move, "moveCharacter")
        
    def move(self, task): 
        
        curtime = task.time
        dt = curtime - self.prevtime 
        self.prevtime = curtime
        
        if self.keyMap["forward"]: 
            self.rig.setY(self.rig, +dt*8)
        if (self.keyMap["backward"]):
            self.rig.setY(self.rig, -dt*8)
        
        if self.keyMap["left"]:  
            self.rig.setH(self.rig.getH() + dt*80)
        if self.keyMap["right"]:
            self.rig.setH(self.rig.getH() - dt*80)
            
        return Task.cont
    
    def setKey(self, key, value): 
        self.keyMap[key] = value
        
    def setupEnv(self):
        # Use myGeom class to create geometry
        def placeCube(center, scale, path = None, name = None):
            self.geom = myGeom(center, scale, smooth=False, tangents=True, mapping="tile")
            self.geom.build() # Build the geometry.
            self.box = render.attachNewNode(self.geom.geomnode)
            # Use myTextureStage class to access multi-textured geom Texture data
            for ts in self.geom.textureStages:            
                if ts.getMode() == TextureStage.MNormal:                
                    ts.setTexture(self.texts.normal_map)
                else:
                    index = self.geom.colors.index(ts.getMatname())
#                     if path is None:
                    tex = self.texts.images[self.texts.colornames[index]]
#                     else:
                        #tex = myTextures(path=path, names=name)
#                         data = Image.open('res/checkerboard.png')
#                         # send data over network or any other place and pass it on
#                         p = PNMImage()
#                         p.read(StringStream(data))
#                         tex = Texture()
#                         tex.load(p)
                        #tex = tex.make_PNMImage('test_bump', r=1,g=1,b=1)   
                    #print(ts.getMatname(), self.texts.colornames[index])
                    ts.setTexture(tex)

                self.box.setTexScale(ts.getTextureStage(), 0.5, 0.5, 0)
            material = Material()
            material.setShininess(1.0)
            self.box.setMaterial(material)

        filename = 'res/env.json'
        with open(filename, 'r') as f:
            env = json.load(f)  
        self.boxHandlers = []
        for i, item in enumerate(env["cubes"]):
#             if "path" in item:
#                 placeCube(center = item["center"], scale = item["scale"], path = item["path"], name = item["name"])
#             else:
            placeCube(center = item["center"], scale = item["scale"])
    def setupFisheye(self):

        self.rig = self.camera.attachNewNode("rig")
        self.camera.setPos(0,0,0)
        self.camera.setHpr(0,0,0)
        x,y,z = 0, 0, 6.1#0,-0.3,0.17
        self.rig.setPos(x,y,z)
        h, p ,r = 0, 0, 0#-0.3, -24, 0.4
        self.rig.setHpr(h,p,r)

        bufferSize = 1024
        buffer = self.win.makeCubeMap("test", bufferSize, self.rig)
        assert buffer

        # we now get buffer thats going to hold the texture of our new scene
        self.altBuffer = self.win.makeTextureBuffer("env", bufferSize, bufferSize, to_ram = True)
        # now we have to setup a new scene graph to make this scene
        altRender = NodePath("new render")

        # altCam on altRender
        altCam = self.makeCamera2d(self.altBuffer)
        altCam.reparentTo(altRender)

        # make fisheye node on altRender
        numVertices = 10000
        fm = FisheyeMaker('card')
        fm.setNumVertices(numVertices)
        fm.setSquareInscribed(True, 1)
        fm.setReflection(True)
        fm.setFov(359.999)

        card = altRender.attachNewNode(fm.generate())
        card.setTexture(buffer.getTexture())
        #altCam.lookAt(card)

        # Disable the scene render on the normal 'render' graph.
        self.win.getDisplayRegion(1).setActive(False)
        finalCard = self.loader.loadModel('res/fisheye.egg')
        finalCard.reparentTo(aspect2d)
        finalCard.setTexture(self.altBuffer.getTexture())
        finalCard.setP(90)   
    def setupLight(self):
        pass
#         pntLight = PointLight("point")
#         pntLight.setColor(Vec4(1.0, 1.0, 1.0, 1.0))
#         #pntLight.setAttenuation((1, 0, 0.01))
#         pntNode = render.attachNewNode(pntLight)
#         pntNode.setPos(0, 2, 3)
#         pntNode.reparentTo(self.cam)
#         render.setLight(pntNode)

#         ambLight = AmbientLight("ambient")
#         ambLight.setColor(Vec4(0.4, 0.4, 0.4, 0.5))
#         ambNode = render.attachNewNode(ambLight)
#         render.setLight(ambNode)
        
if __name__ == "__main__":
    gameApp = Application()
    gameApp.run()        


SystemExit: 